In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.transforms import ToTensor, Compose, Grayscale, Resize, CenterCrop
import matplotlib.pyplot as plt
from PIL import Image
from parse import parse
from custom_loader import AgeDBDataset
from custom_loss_functions import AngularPenaltySMLoss
from tqdm import tqdm
import pretrainedmodels

import warnings
warnings.filterwarnings("ignore")

In [2]:
# hyper params
num_of_class = 79
hidden_unit = 256
learning_rate = 1e-04
batch_size = 64
device = torch.device("cuda")

In [3]:
dataset = AgeDBDataset(
    directory = 'cropped_face/',
    transform = Compose([
        Resize(size=(64,64)),
        CenterCrop(size=64),
        Grayscale(num_output_channels=1),
        ToTensor(),
    ]),
    device = device,
)

In [4]:
len(dataset)

15508

In [5]:
train_set, validation_set, test_set = dataset.get_loaders(
    batch_size=batch_size,
    train_size=0.8,
    test_size=0.2,
)

In [6]:
model = pretrainedmodels.resnet34(pretrained='imagenet')

In [7]:
model.last_linear = nn.Linear(in_features=model.last_linear.in_features, out_features=num_of_class, bias=True)
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
convModel = model.to(device)

In [9]:
# Training loop
def train(model, optimizer, criterion, train_loader, num_of_epoch):
    total_step = len(train_loader)
    for epoch in range(num_of_epoch):
        for i, (imgs, labels) in (enumerate(train_loader)):
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1)%total_step == 0:
                print(f"Epoch: {epoch+1}/{num_of_epoch}, Step: {i+1}/{total_step}, Loss: {loss.item()}")

In [10]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convModel.parameters(), lr=learning_rate)

In [11]:
train(convModel, optimizer, criteria, train_set, num_of_epoch=20)

Epoch: 1/20, Step: 194/194, Loss: 3.9981446266174316
Epoch: 2/20, Step: 194/194, Loss: 3.6836931705474854
Epoch: 3/20, Step: 194/194, Loss: 3.4219210147857666
Epoch: 4/20, Step: 194/194, Loss: 2.4951212406158447
Epoch: 5/20, Step: 194/194, Loss: 1.6892428398132324
Epoch: 6/20, Step: 194/194, Loss: 1.0772026777267456
Epoch: 7/20, Step: 194/194, Loss: 0.9554534554481506
Epoch: 8/20, Step: 194/194, Loss: 0.5537046194076538
Epoch: 9/20, Step: 194/194, Loss: 0.3038618862628937
Epoch: 10/20, Step: 194/194, Loss: 0.21845434606075287
Epoch: 11/20, Step: 194/194, Loss: 0.0807916596531868
Epoch: 12/20, Step: 194/194, Loss: 0.10227324068546295
Epoch: 13/20, Step: 194/194, Loss: 0.048086173832416534
Epoch: 14/20, Step: 194/194, Loss: 0.03199392929673195
Epoch: 15/20, Step: 194/194, Loss: 0.03160897269845009
Epoch: 16/20, Step: 194/194, Loss: 0.015060994774103165
Epoch: 17/20, Step: 194/194, Loss: 0.01650070771574974
Epoch: 18/20, Step: 194/194, Loss: 0.014083893038332462
Epoch: 19/20, Step: 194/19

In [12]:
# Evaluation
def eval(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        error = torch.zeros(0).to(device)
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            outputs = model(imgs)
            
            _, pred = torch.max(outputs.data, 1)

            error = torch.cat([error, torch.abs(
                torch.subtract(torch.reshape(labels, (-1,)), torch.reshape(pred, (-1,)))
            )])
            
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            
    print(f"Accuracy: {(100*correct)/total}%")
    print(f"Mean Absolute Error: {(torch.mean(error))}%")
    print(f"Minimum: {torch.min(error)}, Maximum: {torch.max(error)}, Median: {torch.median(error)}")

In [13]:
eval(convModel, test_set)

Accuracy: 3.8374717832957113%
Mean Absolute Error: 12.54272747039795%
Minimum: 0.0, Maximum: 57.0, Median: 10.0


In [14]:
eval(convModel, train_set)

Accuracy: 81.39609866193777%
Mean Absolute Error: 2.1946637630462646%
Minimum: 0.0, Maximum: 55.0, Median: 0.0
